# **Stock-Price-Scraper**

In [13]:
google_sheet_name = "example stock spreadsheet"      # <------ filename (google spreadsheet)
sheet_name = "stock price"                 # <------ sheet name

from bs4 import BeautifulSoup
import requests
from google.colab import drive
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
data = {}

# get stock names
gc = gspread.authorize(GoogleCredentials.get_application_default())

try:
  sh = gc.open(google_sheet_name)
  wks = sh.worksheet(sheet_name)  
except:
  raise Exception("Spreadsheet " + google_sheet_name + "/"+ sheet_name +" not found.")

values_list = wks.col_values(1)
name = values_list[1:]


# get data
for n in name:
  head = []

  link = "https://hk.finance.yahoo.com/quote/" + n + "/history?p=" + n
  page = requests.get(link)
  if (page.status_code == 404):
    raise Exception("Invalid link or stock name: " + n)
  soup = BeautifulSoup(page.content,'html.parser')

  try:
    thead = soup.find("thead")
    thead_tr = thead.find("tr")
    th = thead_tr.find_all("th")

    tbody = soup.find("tbody")
    tbody_tr = tbody.find("tr")
    td = tbody_tr.find_all("td")
  except:
    raise Exception("Invalid link or stock name: " + n)

  if (head != [] and (th[0].text not in head or th[5].text not in head)):
    raise Exception("header difference:" + th[0].text + " " + th[5].text)
  elif (head == []):
    head = [th[0].text,th[5].text]

  data[n] = [td[0].text,td[5].text]


#print data
print("name" + "\t" + head[0] + '\t\t' + head[1])
for n in name:
  print(n + "\t" + data[n][0] + "\t" + data[n][1])
print()


#write data to spreadsheet
print("\nupdating file: " + google_sheet_name +"\n") 
for n in name:
  try:
    cell = wks.find(n)
  except:
    raise Exception("Cell with name " + n + " not found !!" )
    
  print("Found stock name at Row%s Col%s" % (cell.row, cell.col))
  wks.update_cell(cell.row,cell.col+1, data[n][1])
  print("Wrote stock price at Row%s Col%s" % (cell.row, cell.col+1))

name	日期		經調整收市價**
0001.HK	2020年8月17日	52.350
S68.SI	2020年8月17日	8.64
GOOG	2020年8月14日	1,507.73
7974.T	2020年8月17日	51,550.00


updating file: example stock spreadsheet

Found stock name at Row2 Col1
Wrote stock price at Row2 Col2
Found stock name at Row3 Col1
Wrote stock price at Row3 Col2
Found stock name at Row4 Col1
Wrote stock price at Row4 Col2
Found stock name at Row5 Col1
Wrote stock price at Row5 Col2


*Ref:*

https://realpython.com/beautiful-soup-web-scraper-python/

https://codeburst.io/how-to-scrape-yahoo-finance-using-python-31164aa06468

https://colab.research.google.com/notebooks/io.ipynb#scrollTo=D78AM1fFt2ty